# 清洗数据

数据清洗内容非常庞杂。每个数据集都有自己的情况，可能涉及到缺失值、数据项重复、数据录入错误等等。在本练习中，你将清洗世界银行项目数据集和世界银行指标数据集。 

目前，项目数据集和指标数据集的国家名称是不一致的。你的任务是清洗两个数据集，让它们的国家名称保持一致。这样就可以把两个数据集连接起来了。清洗数据是一项枯燥且繁琐的工作，会占据数据科学家很多时间。

为什么想要把这两个数据集连接起来呢？比如，你想要做一个线性回归，基于指标数据预测项目的成本。或者，你想要分析审批通过的项目的类型和指标数据的关系，或者，研究农村地区电力供应率低的国家是不是会有更多农村类型的项目？

# 第一部分 - 数据探索

运行下面的代码来导入数据集。

In [ ]:
import pandas as pd

# read in the population data and drop the final column
df_indicator = pd.read_csv('../data/population_data.csv', skiprows=4)
df_indicator.drop(['Unnamed: 62'], axis=1, inplace=True)

# read in the projects data set with all columns type string
df_projects = pd.read_csv('../data/projects_data.csv', dtype=str)
df_projects.drop(['Unnamed: 56'], axis=1, inplace=True)

下一个单元格的代码的输出结果是人口指标数据集里去重的国家名称和 ISO 简称。你会注意到有些值表示世界上的一些地区，比如 'East Asia & Pacific' 和 'East Asia & Pacific (excluding high income)'.

In [ ]:
df_indicator[['Country Name', 'Country Code']].drop_duplicates()

在下一个代码单元格里，你会看到项目数据集里去重的国家名称。注意项目数据有两列都是国家名称。一个是国家名称 'countryname'，另一个是国家 'Country'。国家 'Country' 这一列全是 NaN 。

还要注意一点：如果项目数据集和指标数据集一样用的是 [ISO country abbreviations](https://en.wikipedia.org/wiki/ISO_3166-1)，数据集的合并会简单很多。但是，项目数据没有 ISO 国家代号。要将两个数据集合并起来，你有两个选择：
* 向项目数据集加入一列 ISO-3 代号 
* 找到项目数据集和指标数据集国家名称一列的区别，然后清洗数据，让他们保持一致。

运行下方的代码，看看项目数据集的国家名称一列的值。

In [ ]:
df_projects['countryname'].unique()

# 第二部分 - 使用 Pycountry 库

你注意到项目数据集的国家名称有什么模式了吗？每项都有重复，且被分号分隔开：
```text
'Kingdom of Spain;Kingdom of Spain'
'New Zealand;New Zealand'
```

第一步是清洗国家名称，除去分号。在下面实现：

In [ ]:
### 
#
# TODO: In the df_projects dataframe, create a new column called 'Official Country Name' so that the country name only appears once. 
# For example, `Republic of Malta;Republic of Malta` should be `Republic of Malta`.
#
# HINT: use the split() method - see https://pandas.pydata.org/pandas-docs/stable/text.html for examples
# HINT: with pandas, you can do all of this with just one line of code
###

df_projects['Official Country Name'] = None

看起来项目数据集中国家名称是官方的。因此，这个数据集有一项是 "Kingdom of Spain" 而指标数据集只是 "Spain".

幸运的是，有人开发了一个 Python 库，叫做 [pycountry](https://pypi.org/project/pycountry/)。这个库有国家名称、ISO 简称和官方的国家名称。尽管你可能不能用这个库完成所有的国家名称的清洗，但它还是有帮助的。 

运行下方单元格中的代码，安装 pycountry 库，然后观察效果。

In [ ]:
# Run this code cell to install and import the pycountry library
!pip install pycountry
from pycountry import countries

In [ ]:
# Run this code cell to see an example of how the library works
countries.get(name='Spain')

In [ ]:
# Run this code cell to see how you can also look up countries without specifying the key
countries.lookup('Kingdom of Spain')

目标是将 ISO 代号加到项目数据集中。第一步，用 pycountry 库创建一个词典，将去重的官方国家名称 'Official Country Name' 映射到 ISO 代号。

遍历去重的df_projects['Official Country Name'] 列。创建一个词典，将去重的国家名称 'Country Name' 映射到alpha_3 ISO 代号。 

这个词典形式如下：
`{'Kingdom of Spain':'ESP'}`

如果有国家名称在 pycountry 库中没有找到，将它加入列表 `country_not_found`。

In [ ]:
# set up the libraries and variables
from collections import defaultdict
country_not_found = [] # stores countries not found in the pycountry library
project_country_abbrev_dict = defaultdict(str) # set up an empty dictionary of string values

# TODO: iterate through the country names in df_projects. 
# Create a dictionary mapping the country name to the alpha_3 ISO code
for country in df_projects['Official Country Name'].drop_duplicates().sort_values():
    try: 
        # TODO: look up the country name in the pycountry library
        # store the country name as the dictionary key and the ISO-3 code as the value
        project_country_abbrev_dict[country] = None
    except:
        # If the country name is not in the pycountry library, then print out the country name
        # And store the results in the country_not_found list
        print(country, ' not found')
        country_not_found.append(country)

很多国家名称在 pycountry 库中都没有。而且某些名称是地区，如 "South Asia" 或 "Southern Africa"，所以这些地区 pycountry 没有也可以理解。

# 第三部分 - 手动处理

也许这些没有找到的 df_projects 的国家已经包含在指标数据集里了。在下一个单元格，看看 country_not_found 列表是否有指标数据集中的国家。

In [ ]:
# Run this code cell to 
# iterate through the country_not_found list and check if the country name is in the df_indicator data set
indicator_countries = df_indicator[['Country Name', 'Country Code']].drop_duplicates().sort_values(by='Country Name')

for country in country_not_found:
    if country in indicator_countries['Country Name'].tolist():
        print(country)

可惜 df_indicator 和 df_projects 之间匹配的国家名称并不多。这就是数据清洗特别枯燥的地方，尤其在本例中，我们已经帮你做了很多工作。

我们手动创建了一个词典，将 country_not_found 中的国家映射到 ISO-3 alpha 代号。你 **可以** 试着用代码编写一些复杂的字符串匹配算法。这对于大数据来说是节省时间的。但在这种情况下，直接写一个词典可能更快一点。

In [ ]:
# run this code cell to load the dictionary

country_not_found_mapping = {'Co-operative Republic of Guyana': 'GUY',
             'Commonwealth of Australia':'AUS',
             'Democratic Republic of Sao Tome and Prin':'STP',
             'Democratic Republic of the Congo':'COD',
             'Democratic Socialist Republic of Sri Lan':'LKA',
             'East Asia and Pacific':'EAS',
             'Europe and Central Asia': 'ECS',
             'Islamic  Republic of Afghanistan':'AFG',
             'Latin America':'LCN',
              'Caribbean':'LCN',
             'Macedonia':'MKD',
             'Middle East and North Africa':'MEA',
             'Oriental Republic of Uruguay':'URY',
             'Republic of Congo':'COG',
             "Republic of Cote d'Ivoire":'CIV',
             'Republic of Korea':'KOR',
             'Republic of Niger':'NER',
             'Republic of Kosovo':'XKX',
             'Republic of Rwanda':'RWA',
              'Republic of The Gambia':'GMB',
              'Republic of Togo':'TGO',
              'Republic of the Union of Myanmar':'MMR',
              'Republica Bolivariana de Venezuela':'VEN',
              'Sint Maarten':'SXM',
              "Socialist People's Libyan Arab Jamahiriy":'LBY',
              'Socialist Republic of Vietnam':'VNM',
              'Somali Democratic Republic':'SOM',
              'South Asia':'SAS',
              'St. Kitts and Nevis':'KNA',
              'St. Lucia':'LCA',
              'St. Vincent and the Grenadines':'VCT',
              'State of Eritrea':'ERI',
              'The Independent State of Papua New Guine':'PNG',
              'West Bank and Gaza':'PSE',
              'World':'WLD'}

接下来，用新的值更新 project_country_abbrev_dict 变量。

In [ ]:
# TODO: Update the project_country_abbrev_dict with the country_not_found_mapping dictionary
# HINT: This is relatively straightforward. Python dictionaries have a method called update(), which essentially
# appends a dictionary to another dictionary

project_country_abbrev_dict = None

# 第五部分 - 创建 'Country Code' 列

然后，在 data_projects dataframe 中新建一列 'Country Code' 。使用 project_country_abbrev_dict 和列 df_projects['Country Name'] 来新建一列'Country 'Code'。

In [ ]:
# TODO: Use the project_country_abbrev_dict and the df_projects['Country Name'] column to make a new column
# of the alpha-3 country codes. This new column should be called 'Country Code'.

# HINT: Use the apply method and a lambda function
# HINT: The lambda function will use the project_country_abbrev_dict that maps the country name to the ISO code
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html

df_projects['Country Code'] = None

In [ ]:
# Run this code cell to see which projects in the df_projects data frame still have no country code abbreviation.
# In other words, these projects do not have a matching population value in the df_indicator data frame.
df_projects[df_projects['Country Code'] == '']

你会注意到有几行还是没有国家代号。其中几个项目，它的标签是 "Africa"，而不是某个国家。还有 "Yugoslavia"，这个国家在上世纪90年代就不存在了。

# 结语

现在 df_projects dataframe 和 df_indicator dataframe 都有一个一致的列 'Country Code'。但是这两个数据框还不能合并。 

数据框 df_projects 中的每个项目都有一个对应的日期。我们想将 df_projects 和 df_indicator 合并起来，这样的话每个项目都能有一个对应的人口数据。要实现这个目标，还有一些数据转换的工作要做。 

实际上，上一个练习中关于合并数据的、标记为 **困难** 问题可供参考。在那个练习中，指标数据从[宽格式变成长格式](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html)。

你可以继续将 df_projects dataframe 和 df_indicator dataframe，根据 alpha-3 国家代号和项目名称 (project) 或指标的年份 (indicator year) 合并起来。你会发现，数据转换是把数据从一种格式转换为另一种格式的一系列流程。
